# Lab 4: Urban Heat Island Detection using Thermal Satellite Imagery

## Goal
In this exercise, you will:
1. Select and compare two thermal satellite images from Kraków (or another city that has recently undergone intensive, thoughtless concrete development) – one from **2013** and one from **2024**.
2. Each image must be acquired on a **hot summer day** (T > 27°C based on IMGW meteorological data).
3. Images must have **low cloud cover** (< 20%).
4. Visualize thermal data and analyze surface temperature differences (Urban Heat Island effect).

In [2]:
import ee
import geemap
# ee.Authenticate()
ee.Initialize(project="ee-zuzannaslobodzian2")

## Task 1: Area of Interest (AOI)
- Define an AOI over the Reduta Street area in Kraków.
- Use `ee.Geometry.Polygon` or `ee.Geometry.Rectangle`.

In [3]:
aoi = ee.Geometry.Rectangle([19.884968, 50.025676, 19.908652, 50.012329])

## Task 2: Download and Analyze IMGW Meteorological Data
- Visit: [IMGW Archive](https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/)
- Navigate to:
  - `dane_meteorologiczne/dobowe/synop/2013/`
  - `dane_meteorologiczne/dobowe/synop/2024/`
- Download `s_d_tmax.csv` for both years.
- Filter the rows for **station ID 12566 (e.g. Kraków-Balice)**.
- Identify days in **July or August** with **TMAX > 27°C**.

In [47]:
import pandas as pd
df = pd.read_csv('C:\\space_technologies\\remote_sensing\\rs-summer-2025-labs-ZuzannaSlobodzian\\data\\k_d_07_2013.csv', encoding='cp1250')

df.columns = [
    'id', 'miejsce', 'rok', 'miesiac', 'dzien', 'temperatura',
    'kol1', 'kol2', 'kol3', 'kol4', 'kol5', 'kol6',
    'kol7', 'kol8', 'kol9', 'kol10', 'kol11', 'kol12'
]

# Zapisz z nagłówkami (opcjonalnie)
df.to_csv('2013_ok.csv', index=False)

df = pd.read_csv('C:\\space_technologies\\remote_sensing\\rs-summer-2025-labs-ZuzannaSlobodzian\\data\\k_d_08_2024.csv', encoding='cp1250')

df.columns = [
    'id', 'miejsce', 'rok', 'miesiac', 'dzien', 'temperatura',
    'kol1', 'kol2', 'kol3', 'kol4', 'kol5', 'kol6',
    'kol7', 'kol8', 'kol9', 'kol10', 'kol11', 'kol12'
]

# Zapisz z nagłówkami (opcjonalnie)
df.to_csv('2024_ok.csv', index=False)

In [48]:
import pandas as pd

# 250190390
df = pd.read_csv('C:\\space_technologies\\remote_sensing\\rs-summer-2025-labs-ZuzannaSlobodzian\\notebooks\\2013_ok.csv',  encoding='latin1')
print(df.columns)

hot_days_2013 = df[df['id'] == 250190390]

hot_days_2013 = hot_days_2013[hot_days_2013['temperatura'] > 27]

print("hot_days_2013")
print(hot_days_2013)

df = pd.read_csv('C:\\space_technologies\\remote_sensing\\rs-summer-2025-labs-ZuzannaSlobodzian\\notebooks\\2024_ok.csv',  encoding='latin1')
print(df.columns)

hot_days_2024 = df[df['id'] == 250190390]

hot_days_2024 = hot_days_2024[hot_days_2024['temperatura'] > 27]

print("hot_days_2024")
print(hot_days_2024)


Index(['id', 'miejsce', 'rok', 'miesiac', 'dzien', 'temperatura', 'kol1',
       'kol2', 'kol3', 'kol4', 'kol5', 'kol6', 'kol7', 'kol8', 'kol9', 'kol10',
       'kol11', 'kol12'],
      dtype='object')
hot_days_2013
             id                miejsce   rok  miesiac  dzien  temperatura  \
2171  250190390  KRAKÃW-OBSERWATORIUM  2013        7      3         29.2   
2172  250190390  KRAKÃW-OBSERWATORIUM  2013        7      4         28.9   
2173  250190390  KRAKÃW-OBSERWATORIUM  2013        7      5         27.6   
2177  250190390  KRAKÃW-OBSERWATORIUM  2013        7      9         27.8   
2178  250190390  KRAKÃW-OBSERWATORIUM  2013        7     10         29.0   
2186  250190390  KRAKÃW-OBSERWATORIUM  2013        7     18         27.8   
2187  250190390  KRAKÃW-OBSERWATORIUM  2013        7     19         28.5   
2190  250190390  KRAKÃW-OBSERWATORIUM  2013        7     22         29.0   
2191  250190390  KRAKÃW-OBSERWATORIUM  2013        7     23         27.1   
2194  25019039

## Task 3: Select Landsat 8 Images Matching These Dates
- In Earth Engine, use `LANDSAT/LC08/C02/T1_L2` collection.
- Apply filters:
  - `.filterBounds(aoi)`
  - `.filterDate()` for the matching day
  - `.filterMetadata('CLOUD_COVER', 'less_than', 20)`
- Try to find **one image from 2013** and **one image from 2024** that match your hot days list.

In [51]:
# Pobierz kolekcję (bez .first())
collection_2013 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(aoi)
    .filterDate('2013-07-01', '2013-07-31')
    .filterMetadata('CLOUD_COVER', 'less_than', 20)
)

# Zamień na listę (max 50 obrazów)
image_list = collection_2013.toList(50)

# Wyciągnij datę i zachmurzenie dla każdego obrazu
for i in range(image_list.size().getInfo()):
    img = ee.Image(image_list.get(i))
    props = img.toDictionary(['DATE_ACQUIRED', 'CLOUD_COVER']).getInfo()
    
    print(f"{i+1}. Data: {props['DATE_ACQUIRED']}, Zachmurzenie: {props['CLOUD_COVER']}%")
    
    
image_2013 = collection_2013.first()

# Pobierz kolekcję (bez .first())
collection_2024 = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    .filterBounds(aoi)
    .filterDate('2024-08-01', '2024-08-31')
    .filterMetadata('CLOUD_COVER', 'less_than', 20)
)

# Zamień na listę (max 50 obrazów)
image_list = collection_2024.toList(50)

# Wyciągnij datę i zachmurzenie dla każdego obrazu
for i in range(image_list.size().getInfo()):
    img = ee.Image(image_list.get(i))
    props = img.toDictionary(['DATE_ACQUIRED', 'CLOUD_COVER']).getInfo()
    
    print(f"{i+1}. Data: {props['DATE_ACQUIRED']}, Zachmurzenie: {props['CLOUD_COVER']}%")
    
image_2024 = collection_2024.first()

image_2013 = image_2013.clip(aoi)
image_2024 = image_2024.clip(aoi)

Map = geemap.Map()
Map.centerObject(aoi, zoom=15)

# RGB z pasm SR (scaled reflectance)
vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 5000,
    'max': 15000,
    'gamma': 1.3
}

Map.addLayer(image_2013, vis_params, 'Landsat 8 - 2013')
Map.addLayer(image_2024, vis_params, 'Landsat 9 - 2024')

Map.addLayer(aoi, {}, "AOI")
Map

1. Data: 2013-07-22, Zachmurzenie: 8.95%
1. Data: 2024-08-13, Zachmurzenie: 0.21%
2. Data: 2024-08-29, Zachmurzenie: 0.49%


Map(center=[50.01900279353348, 19.896809999996524], controls=(WidgetControl(options=['position', 'transparent_…

## Task 4: Process Thermal Band (ST_B10)
- Convert Band 10 to Brightness Temperature (Kelvin):
  `TB = ST_B10 * 0.00341802 + 149.0`
- Create a visualization of each image using the same color scale.

In [55]:
tb_2013 = image_2013.select('ST_B10').multiply(0.00341802).add(149.0).rename('Brightness_Temperature')
tb_2024 = image_2024.select('ST_B10').multiply(0.00341802).add(149.0).rename('Brightness_Temperature')

# Zdefiniuj parametry wizualizacji
vis_params = {
    'min': 290,  # ~17°C
    'max': 320,  # ~47°C
    'palette': [
    '#ffffcc', '#ffeda0', '#fed976', '#feb24c',
    '#fd8d3c', '#fc4e2a', '#e31a1c', '#bd0026', '#800026'
]
}

# Wyświetl na mapie
Map = geemap.Map()
Map.centerObject(aoi, 15)
Map.addLayer(tb_2013, vis_params, '2013')
Map.addLayer(tb_2024, vis_params, '2024')
Map


Map(center=[50.01900279353348, 19.896809999996524], controls=(WidgetControl(options=['position', 'transparent_…

## Task 5: Compare and Interpret
- Compare the two maps.
- Optionally calculate difference: `TB_2024 - TB_2013`
- Discuss: did the surface temperature increase in the area?
- Is there evidence of an Urban Heat Island effect related to development?

The 2024 thermal map shows a noticeable increase in surface temperature compared to 2013, with more widespread red areas indicating higher heat levels. This suggests that the area has experienced warming over the years, likely due to increased urban development. The clear concentration of heat in built-up zones provides strong evidence of an Urban Heat Island effect linked to land cover changes.